# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6b8c016850>
├── 'a' --> tensor([[-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848]])
└── 'x' --> <FastTreeValue 0x7f6b8c016c70>
    └── 'c' --> tensor([[ 0.1309, -0.0276, -0.0228,  1.7603],
                        [-0.4303,  0.6171, -0.7754, -0.6983],
                        [-2.3598, -0.5060,  0.8833,  0.7631]])

In [4]:
t.a

tensor([[-1.5544, -0.3811, -0.2430],
        [-0.2267, -1.1976,  1.8848]])

In [5]:
%timeit t.a

73.8 ns ± 0.848 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6b8c016850>
├── 'a' --> tensor([[-0.3433, -1.8067, -1.1069],
│                   [-1.6108,  0.9416,  0.7182]])
└── 'x' --> <FastTreeValue 0x7f6b8c016c70>
    └── 'c' --> tensor([[ 0.1309, -0.0276, -0.0228,  1.7603],
                        [-0.4303,  0.6171, -0.7754, -0.6983],
                        [-2.3598, -0.5060,  0.8833,  0.7631]])

In [7]:
%timeit t.a = new_value

76 ns ± 1.42 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5544, -0.3811, -0.2430],
               [-0.2267, -1.1976,  1.8848]]),
    x: Batch(
           c: tensor([[ 0.1309, -0.0276, -0.0228,  1.7603],
                      [-0.4303,  0.6171, -0.7754, -0.6983],
                      [-2.3598, -0.5060,  0.8833,  0.7631]]),
       ),
)

In [10]:
b.a

tensor([[-1.5544, -0.3811, -0.2430],
        [-0.2267, -1.1976,  1.8848]])

In [11]:
%timeit b.a

68.9 ns ± 0.83 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2311, -1.0229, -0.2293],
               [ 0.3359, -0.6618, -0.9775]]),
    x: Batch(
           c: tensor([[ 0.1309, -0.0276, -0.0228,  1.7603],
                      [-0.4303,  0.6171, -0.7754, -0.6983],
                      [-2.3598, -0.5060,  0.8833,  0.7631]]),
       ),
)

In [13]:
%timeit b.a = new_value

630 ns ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.12 µs ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.7 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

294 µs ± 6.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

293 µs ± 4.48 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6ae88f6040>
├── 'a' --> tensor([[[-1.5544, -0.3811, -0.2430],
│                    [-0.2267, -1.1976,  1.8848]],
│           
│                   [[-1.5544, -0.3811, -0.2430],
│                    [-0.2267, -1.1976,  1.8848]],
│           
│                   [[-1.5544, -0.3811, -0.2430],
│                    [-0.2267, -1.1976,  1.8848]],
│           
│                   [[-1.5544, -0.3811, -0.2430],
│                    [-0.2267, -1.1976,  1.8848]],
│           
│                   [[-1.5544, -0.3811, -0.2430],
│                    [-0.2267, -1.1976,  1.8848]],
│           
│                   [[-1.5544, -0.3811, -0.2430],
│                    [-0.2267, -1.1976,  1.8848]],
│           
│                   [[-1.5544, -0.3811, -0.2430],
│                    [-0.2267, -1.1976,  1.8848]],
│           
│                   [[-1.5544, -0.3811, -0.2430],
│                    [-0.2267, -1.1976,  1.8848]]])
└── 'x' --> <FastTreeValue 0x7f6ae1cab4c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44.9 µs ± 907 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6b8c08c130>
├── 'a' --> tensor([[-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848],
│                   [-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848],
│                   [-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848],
│                   [-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848],
│                   [-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848],
│                   [-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848],
│                   [-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848],
│                   [-1.5544, -0.3811, -0.2430],
│                   [-0.2267, -1.1976,  1.8848]])
└── 'x' --> <FastTreeValue 0x7f6ae1cab340>
    └── 'c' --> tensor([[ 0.1309, -0.0276, -0.0228,  1.7603],
                        [-0.4303,  0.6171, -0.7754, -0.6983],
                 

In [23]:
%timeit t_cat(trees)

40.8 µs ± 552 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.9 µs ± 2.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.1309, -0.0276, -0.0228,  1.7603],
                       [-0.4303,  0.6171, -0.7754, -0.6983],
                       [-2.3598, -0.5060,  0.8833,  0.7631]],
              
                      [[ 0.1309, -0.0276, -0.0228,  1.7603],
                       [-0.4303,  0.6171, -0.7754, -0.6983],
                       [-2.3598, -0.5060,  0.8833,  0.7631]],
              
                      [[ 0.1309, -0.0276, -0.0228,  1.7603],
                       [-0.4303,  0.6171, -0.7754, -0.6983],
                       [-2.3598, -0.5060,  0.8833,  0.7631]],
              
                      [[ 0.1309, -0.0276, -0.0228,  1.7603],
                       [-0.4303,  0.6171, -0.7754, -0.6983],
                       [-2.3598, -0.5060,  0.8833,  0.7631]],
              
                      [[ 0.1309, -0.0276, -0.0228,  1.7603],
                       [-0.4303,  0.6171, -0.7754, -0.6983],
                       [-2.3598, -0.5060,  0.8833,  0.7631]],

In [26]:
%timeit Batch.stack(batches)

101 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.1309, -0.0276, -0.0228,  1.7603],
                      [-0.4303,  0.6171, -0.7754, -0.6983],
                      [-2.3598, -0.5060,  0.8833,  0.7631],
                      [ 0.1309, -0.0276, -0.0228,  1.7603],
                      [-0.4303,  0.6171, -0.7754, -0.6983],
                      [-2.3598, -0.5060,  0.8833,  0.7631],
                      [ 0.1309, -0.0276, -0.0228,  1.7603],
                      [-0.4303,  0.6171, -0.7754, -0.6983],
                      [-2.3598, -0.5060,  0.8833,  0.7631],
                      [ 0.1309, -0.0276, -0.0228,  1.7603],
                      [-0.4303,  0.6171, -0.7754, -0.6983],
                      [-2.3598, -0.5060,  0.8833,  0.7631],
                      [ 0.1309, -0.0276, -0.0228,  1.7603],
                      [-0.4303,  0.6171, -0.7754, -0.6983],
                      [-2.3598, -0.5060,  0.8833,  0.7631],
                      [ 0.1309, -0.0276, -0.0228,  1.7603],
                   

In [28]:
%timeit Batch.cat(batches)

189 µs ± 3.42 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

620 µs ± 9.09 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
